Copyright **`(c)`** 2023 Antonio Ferrigno `<s316467@polito.it>`  
[`https://github.com/s316467/Computational-Intelligence-23-24/tree/main`](https://github.com/s316467/Computational-Intelligence-23-24/tree/main)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/s316467/Computational-Intelligence-23-24/tree/main/LICENSE.md) for details.

# LAB9

Wrote a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Friday, December 8 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, November 27
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [11]:
from random import choices, random, randint, sample

import lab2_lib

In [7]:
# Function to create a specific problem instance
def create_problem_instance(instance_number):
    return lab2_lib.make_problem(instance_number)

In [9]:
# Initialize a population of genomes
def initialize_population(size, length):
    return [choices([0, 1], k=length) for _ in range(size)]

# Evaluate the fitness of each genome with optimization
def evaluate_population(population, previous_population, previous_fitness_values):
    new_fitness_values = []
    for i, individual in enumerate(population):
        if not previous_population or individual != previous_population[i]:
            new_fitness_values.append(fitness(individual))
        else:
            new_fitness_values.append(previous_fitness_values[i])
    return new_fitness_values

# Tournament selection
def tournament_selection(population, fitnesses, tournament_size=3):
    selected = []
    for _ in range(2):
        tournament = sample(list(zip(population, fitnesses)), tournament_size)
        selected.append(max(tournament, key=lambda x: x[1])[0])
    return selected

# Perform crossover between two parents
def crossover(parent1, parent2):
    crossover_point = randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Mutate a genome
def mutate(genome):
    for i in range(len(genome)):
        if random() < mutation_rate:
            genome[i] = 1 - genome[i]
    return genome

In [37]:
# Choose the problem instance here
problem_instance_number = 1  # Change this to 1, 2, 5, 10 or 1000 as needed
fitness = create_problem_instance(problem_instance_number)

# Parameters
genome_length = 1000
population_size = 100
max_generations = 1000
mutation_rate = 0.01
convergence_threshold = 0.01  # Threshold for convergence-based termination

# Main Evolutionary Algorithm
population = initialize_population(population_size, genome_length)
previous_population = []
fitness_values = evaluate_population(population, previous_population, [])

best_fitness_history = []

for generation in range(max_generations):
    new_population = []
    for _ in range(population_size // 2):
        parent1, parent2 = tournament_selection(population, fitness_values)
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        new_population.extend([child1, child2])

    fitness_values = evaluate_population(new_population, population, fitness_values)
    population = new_population

    best_fitness = max(fitness_values)
    best_fitness_history.append(best_fitness)

    # Convergence check
    if generation > 0 and abs(best_fitness - best_fitness_history[-2]) < convergence_threshold:
        print(f"Convergence reached at generation {generation}.")
        break

    # Optional: Print best fitness in each generation
    # print(f"Generation {generation}: Best Fitness = {best_fitness}")

# Find and print the best solution
best_fitness = max(fitness_values)
best_genome = population[fitness_values.index(best_fitness)]
# print("Best Genome:", best_genome) # may be too long to print
print("Best Fitness:", best_fitness)
print("Total Fitness Calls:", fitness.calls)

Convergence reached at generation 1.
Best Fitness: 0.565
Total Fitness Calls: 300
